In [1]:
import time
import requests

In [93]:
#enable load racks
r = requests.get('http://131.243.41.59:9090/control_enable')
print(r)

<Response [200]>


In [101]:
import requests
import time
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Time Start =", current_time)

r = requests.get('http://131.243.41.59:9090/control?group_id=0,P_ctrl=0')
print(r)
#time.sleep(600)
# r = requests.get('http://131.243.41.59:9090/control?group_id=0,P_ctrl=0')
# print(r)
# time.sleep(60)

# r = requests.get('http://131.243.41.59:9090/control?group_id=1,P_ctrl=2000')
# print(r)
# time.sleep(600)
# r = requests.get('http://131.243.41.59:9090/control?group_id=1,P_ctrl=0')
# print(r)
# time.sleep(60)

# r = requests.get('http://131.243.41.59:9090/control?group_id=2,P_ctrl=2000')
# print(r)
# time.sleep(600)
# r = requests.get('http://131.243.41.59:9090/control?group_id=2,P_ctrl=0')
# print(r)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Time End =", current_time)

Time Start = 11:21:12
<Response [200]>
Time End = 11:21:13


In [125]:
import numpy as np

def modbustoOpal_quadrant(Pcmd_kVA, Qcmd_kVA, Pact, Qact, act_idxs):
    IP = '131.243.41.14'
    PORT = 504
    id = 3
    inv_1 = 101
    inv_2 = 102
    inv_3 = 103
    act_idxs_registers = []
    pq_changed = []
    for i,j in zip(range(len(act_idxs)), act_idxs): #checks to see if any sign changes occured from last command
        if np.sign(Pcmd_kVA[i]) != np.sign(Pact[i]) or np.sign(Qcmd_kVA[i]) != np.sign(Qact[i]):
            act_idxs_registers.append(j)
            pq_changed.append(i)
    if len(act_idxs_registers) > 0: #if any quadrant changes, execute modbus, else return.
        value = [0] * len(act_idxs_registers)
        inv_act_idxs_registers = act_idxs_registers.copy()
        client = ModbusClient(IP, port=PORT)
        for i in range(len(act_idxs_registers)): # determines which inverters have quadrant change
            if inv_act_idxs_registers[i] == 1:
                inv_act_idxs_registers[i] = inv_1
            elif inv_act_idxs_registers[i] == 2:
                inv_act_idxs_registers[i] = inv_2
            elif inv_act_idxs_registers[i] == 3:
                inv_act_idxs_registers[i] = inv_3
        for i,j in zip(pq_changed, range(len(act_idxs_registers))): # determines exact quadrant for inverter
            if Pcmd_kVA[i] >= 0 and Qcmd_kVA[i] >= 0: #quadrant 1
                value[j] = 1
            if Pcmd_kVA[i] < 0 and Qcmd_kVA[i] >= 0: #quadrant 2
                value[j] = 2
            if Pcmd_kVA[i] < 0 and Qcmd_kVA[i] < 0: #quadrant 3
                value[j] = 3
            if Pcmd_kVA[i] >= 0 and Qcmd_kVA[i] < 0: #quadrant 4
                value[j] = 4
        for i in range(len(act_idxs_registers)): # write quadrant changes to modbus registers
            client.write_registers(inv_act_idxs_registers[i], value[i] , unit = id)
            print('Quadrant change for inv:', inv_act_idxs_registers[i], 'to quadrant', value[i] )
    else:
        return
    
Pcmd_kVA = np.array([-10])
Qcmd_kVA = np.array([10])
Pact = np.array([-10])
Qact = np.array([-10])
act_idxs = np.array([3])
modbustoOpal_quadrant(Pcmd_kVA, Qcmd_kVA, Pact, Qact, act_idxs)

[3]
[103]
[3]
Quadrant change for inv: 103 to quadrant 2


In [ ]:
'test time for modbus quadrant command'

IP = '131.243.41.14'
PORT = 504
id = 3
inv_1 = 101
inv_2 = 102
inv_3 = 103

inv_registers = [1,2,3]
act_registers = [1,2,3,4]

for 
    client = ModbusClient(IP, port=PORT)

    t0=time.time()
    client.write_registers(int(1), int(1) , unit = id)
    print(f'time to execute: {time.time()-t0}')

In [ ]:
def API_inverters(self, act_idxs, Pcmd_kVA, Qcmd_kVA, inv_Pmax, inv_Qmax):
    Pcmd_VA = abs(Pcmd_kVA*1000) #abs values for working only in quadrant 1. Will use modbus to determine quadrant
    Qcmd_VA = abs(Qcmd_kVA*1000) #abs values for working only in quadrant 1. Will use modbus to determine quadrant
    Pcmd_perc = Pcmd_VA/inv_Pmax #Pcmd to inverters must be a percentage of Pmax
    Qcmd_perc = Qcmd_VA/inv_Qmax #Qcmd to inverters must be a percentage of Qmax
    act_idxs = act_idxs.tolist()
    for i in range(len(Pcmd_perc)): # checks Pcmd for inverter limit
        if Pcmd_perc[i] > 100:
            Pcmd_perc[i] = 100
    for j in range(len(Qcmd_perc)): # checks Qcmd for inverter limit
        if Qcmd_perc[j] > 100:
            Qcmd_perc[j] = 100
    for Pcmd_perc_phase, Qcmd_perc_phase, inv in zip(Pcmd_perc, Qcmd_perc, act_idxs):
        Pcmd_perc_phase = Pcmd_perc_phase.item() #changes data type from numpy to python int/float
        Qcmd_perc_phase = Qcmd_perc_phase.item() #changes data type
        inv = inv.item() #changes data type
        flexgrid.set_dyn_P(Pcmd_perc_phase,inv)
        time.sleep(0.2) #pause so modbus does not crash
        flexgrid.set_dyn_Q(Qcmd_perc_phase,inv)
        time.sleep(0.2) #pause so modbus does not crash